In [102]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.linear_model import LinearRegression, datasets
from sklearn import datasets

In [215]:
class linear_regression():
    def __init__(self, x, y, b0, b1, test_data, training_data, test_df, training_df,df,col_names):
        self.x = x
        self.y = y
        self.beta0 = b0
        self.beta1 = b1
        self.trainingdata = training_data
        self.testdata = test_data
        self.trainingdf = training_df
        self.testdf = test_df
        self.df = df
        self.columnnames = col_names
    def describetest(self):
        print(self.testdf.describe())
    def describetraining(self):
        print(self.trainingdf.describe())
    def graph_histogram(self,column_name,from_df):
        try:
            if(from_df == 'Training'):
                print("Training Data Histogram for", format(column_name))
                sns.displot(self.trainingdf[column_name], kde=True,height=6, aspect=2,)
            elif(from_df == 'Test'):
                print("Test Data Histogram for", format(column_name))
                sns.displot(self.testdf[column_name], kde=True,height=6, aspect=2,)
            else:
                print("Data Histogram for", format(column_name))
                sns.displot(self.df[column_name], kde=True,height=6, aspect=2,)
        except:
            print('Error en datos de entrada')
    def graph_correlation(self):
        col_name_list = self.columnnames
        var_to_compare = col_name_list.pop(0)
        for item in col_name_list:
            corr_val = self.df[var_to_compare].corr(self.df[item])
            plt.scatter(self.df[item], self.df[var_to_compare], color = 'hotpink')
            plt.title("Correlacion " +str(var_to_compare)+ " - "+ item + " " + str("%.2f" % corr_val))
            plt.xlabel(item)
            plt.ylabel(var_to_compare)
            plt.show()
    def tupu(self):
        self.x = 1
    @classmethod
    def from_file(cls, filename, training_percent):
        data = np.load(filename)
        training_data_percent = training_percent
        test_data_percent = 100-training_data_percent
        training_data_samples = int(data.shape[0] * (training_data_percent*0.01))
        random_indexes = np.random.choice(data.shape[0], size=training_data_samples, replace=False)
        training_data = data[random_indexes,:]
        test_data = np.delete(data,random_indexes,axis=0)
        columns_names = ['SalePrice','OverallQuality','1stFlrSF','TotRmsAbvGrd','YearBuilt','LotFrontage']
        dftraining = pd.DataFrame(training_data, columns = columns_names)
        dftest = pd.DataFrame(test_data, columns = columns_names)
        df = pd.DataFrame(data, columns = columns_names)
        x=0
        y=0
        beta_0=0
        beta_1=0
        return cls(x,y,beta_0,beta_1,test_data, training_data,dftest,dftraining,df,columns_names)
    

In [216]:
kev = linear_regression.from_file('proyecto_training_data.npy',80)

In [217]:
#kev.graph_histogram('OverallQuality','Training')

In [218]:
#kev.graph_correlation()